<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="./images/btp-banner.gif" alt="BTP A&C">
</div>

## Hybrid RAG with SAP HANA Cloud Vector Engine and Knowledge Graph Engine
In this demo, we will explore how to build hybrid RAG (Retrieval-Augmented Generation) using SAP HANA Cloud Vector Engine and Knowledge Graph Engine. Hybrid RAG is a modern and powerful approach to improving the quality of answers generated by large language models (LLMs). This hybrid approach enables the LLM to generate more accurate, comprehensive, and trustworthy responses, making it especially valuable in enterprise applications, customer support, healthcare, legal, and scientific domains where both nuance and factual accuracy matter. You will learn how to use Vector similarity search and SPARQL queries to retrieve and analyze supplier information from both unstructured and structured data. 

## 🎯Learning Objectives
By the end of this demo, you will be able to:
- Design and implement a dual-retrieval strategy where both vector similarity search and SPARQL queries extract relevant context for a user query.
- Integrate and orchestrate responses by combining vector-retrieved content and knowledge graph-derived metadata to formulate precise and context-aware LLM prompts.

## 🚨Requirements

Please review the following requirements before starting the demo: 
- Complete the notebook **Retrieval-Augmented Generation with SAP HANA Cloud Vector Engine**
- Complete the notebook **Retrieval-Augmented Generation with SAP HANA Cloud Knowledge Graph**

### Step 1: Install Python packages

Run the following package installations. **pip** is the package installer for Python. You can use pip to install packages from the Python Package Index and other indexes.

⚠️**Note:** Jupyter Notebook kernel restart required after package installation.

In [ ]:
%pip install generative-ai-hub-sdk[all] --break-system-packages
%pip install hdbcli --break-system-packages
%pip install langchain-core --break-system-packages
%pip install langchain-hana --break-system-packages
%pip install pandas --break-system-packages
%pip install xml.etree --break-system-packages
%pip install python-dotenv --break-system-packages

# kernel restart required!!!

### Step 2: Configure AI Core Client
Execute the configuration module below to enable access to SAP’s Generative AI foundation models. Running this code block will automatically handle the necessary setup, including authentication and environment configuration, to ensure seamless connectivity to the Generative AI services.

In [ ]:
from ai_core_sdk.ai_core_v2_client import AICoreV2Client
from gen_ai_hub.proxy.gen_ai_hub_proxy import GenAIHubProxyClient
from dotenv import load_dotenv
import os

# Get the AI Core URL from environment variables
URL = os.getenv('AICORE_AUTH_URL')
# Get the AI Core client ID from environment variables
CLIENT_ID = os.getenv('AICORE_CLIENT_ID')
# Get the AI Core client secret from environment variables
CLIENT_SECRET = os.getenv('AICORE_CLIENT_SECRET')
# Get the AI Core client ID from environment variables
RESOURCE_GROUP = os.getenv('AICORE_RESOURCE_GROUP')
# Get the AI Core client secret from environment variables
API_URL = os.getenv('AICORE_BASE_URL')

# Set up the AICoreV2Client
ai_core_client = AICoreV2Client(base_url=API_URL,
                            auth_url=URL,
                            client_id=CLIENT_ID,
                            client_secret=CLIENT_SECRET,
                            resource_group=RESOURCE_GROUP)

# Initialize GenAIHub proxy client
proxy_client = GenAIHubProxyClient(ai_core_client=ai_core_client)
print("✅AI Core Client connection is established successfully!")

Initialize the embedding and LLM models

In [ ]:
# Initialize embedding and LLM models
from gen_ai_hub.proxy.langchain import OpenAIEmbeddings, ChatOpenAI, ChatBedrock

embedding_model = OpenAIEmbeddings(proxy_model_name='text-embedding-ada-002', proxy_client=proxy_client)
gpt_model = ChatOpenAI(proxy_model_name='gpt-4o', proxy_client=proxy_client)
anthropic_model = ChatBedrock(model_name="anthropic--claude-3.5-sonnet", proxy_client=proxy_client)
print("✅AI models are initialized successfully!")

### Step 3: Connect to SAP HANA Cloud database

The provided Python script imports database connection modules and initiates a connection to a SAP HANA Cloud instance using the **dbapi** module. The user is prompted to enter their username and password, which are then used to establish a secure connection to the SAP HANA Cloud database. 

In [ ]:
#Set up HANA Cloud Connection
from hdbcli import dbapi

load_dotenv()  # take environment variables from .env.

# Get the HANA Cloud username from environment variables
HANA_USER = os.getenv('HANA_VECTOR_USER')
# Get the HANA Cloud password from environment variables
HANA_PASS = os.getenv('HANA_VECTOR_PASS')
# Get the HANA Cloud host from environment variables
HANA_HOST = os.getenv('HANA_VECTOR_HOST')

# Establish connection to SAP HANA Cloud database
conn = dbapi.connect(
    user = HANA_USER,
    password = HANA_PASS,
    address = HANA_HOST,
    port = 443,
)
cursor = conn.cursor()

print("✅HANA Cloud connection is established successfully!")

Create a LangChain VectorStore interface for the HANA database and specify the table (collection) to use for accessing the vector embeddings. Embeddings are vector representations of text data that incorporate the semantic meaning of the text.

In [ ]:
from langchain_hana import HanaDB

#Create a LangChain VectorStore interface for the HANA database and specify the table (collection) to use for accessing the vector embeddings
db_vec_table = HanaDB(
    embedding=embedding_model, 
    connection=conn, 
    table_name="PRODUCTS_IT_ACCESSORY_ADA_"+ HANA_USER,
    content_column="VEC_TEXT", # the original text description of the product details
    metadata_column="VEC_META", # metadata associated with the product details
    vector_column="VEC_VECTOR" # the vector representation of each product 
)
print("✅Vector Database connection is established successfully!")

### Step 4: Complex Queries
A sample question that queries both vector database and tabular data from a Knowledge Graph.  

In [ ]:
question = "I want to see all suppliers that sell Logitech keyboard with product rating higher than 4-star, and have a risk score lower than 20."

### Step 5: Retrieve context from SAP HANA Cloud Vector Engine

In [ ]:
# Retrieve vector context
def retrieve_vector_context(question, top_k=25):
    retriever = db_vec_table.as_retriever(search_kwargs={'k': top_k})
    return retriever.invoke(question)

vector_context = retrieve_vector_context(question)
print("Vector Context Retrieved:")
print(vector_context)

### Step 6: Retrieve tabular data from SAP HANA Cloud Knowledge Graph Engine

In [ ]:
from langchain_core.prompts import PromptTemplate
from typing import Dict, List
from xml.etree import ElementTree as ET
import pandas as pd

def extract_metadata(conn) -> List[Dict]:
    """Extract relevant metadata from RDF triples using SPARQL"""
    cursor = conn.cursor()
    
    try:
        # Execute SPARQL query to get all relevant triples
        graph_uri = "spurchase_graph_" + HANA_USER
        sparql_query = """
        SELECT * WHERE {
        GRAPH <""" + graph_uri + """> {
            ?s ?p ?o
        }
        }
        """
        
        resp = cursor.callproc('SPARQL_EXECUTE', (sparql_query, 'Metadata headers describing Input and/or Output', '?', None))
        
        if resp and len(resp) >= 3 and resp[2]:
            # Parse the XML response
            xml_response = resp[2]
            results = parse_sparql_results(xml_response)
            
            # Convert to our standard format
            metadata = []
            for row in results:
                metadata.append({
                    's': row.get('s', ''),
                    'p': row.get('p', ''),
                    'o': row.get('o', '')
                })
            return metadata
        return []
    
    except Exception as e:
        print(f"Error executing SPARQL query: {e}")
        return []
    finally:
        cursor.close()

def analyze_metadata(metadata: List[Dict], question: str, anthropic) -> Dict:
    """Analyze the metadata to identify tables, columns, and relationships"""
    # Convert metadata to a format the LLM can understand
    metadata_str = "\n".join([f"{item['s']} {item['p']} {item['o']}" for item in metadata])
    
    prompt_template = """Given the following RDF metadata about database tables and columns, analyze the user's question and identify:
    1. The main table(s) involved with their schema (SPURCHASE)
    2. The columns needed (including any aggregation functions)
    3. Any filters or conditions
    4. Any joins required

    Important Rules:
    - Always include the schema name (SPURCHASE) before table names
    - When using GROUP BY, include the grouping columns in SELECT
    - Never include any explanatory text in the SQL output
    - For country codes like Germany, use 'DE' in filters
    - Ignore any product names or descriptions
    - Ignore any product ratings or reviews

    For each column, include:
    - The column name (prefix with table alias if needed)
    - Any aggregation function (AVG, COUNT, etc.)
    - Any filter conditions
    - Whether it's a grouping column

    For tables, include:
    - The full table name with schema (e.g., SPURCHASE.S013)
    - Any relationships to other tables


    Metadata:
    {metadata}

    Question: {question}

    Return your analysis in this exact format (without any additional explanations):
    Tables: [schema.table]
    Columns: [column names with aggregations like AVG(RISK1)]
    Filters: [filter conditions]
    Joins: [join conditions]
    GroupBy: [columns to group by]
    """
    
    prompt = PromptTemplate.from_template(prompt_template).invoke({
        "metadata": metadata_str,
        "question": question
    })
    
    # We'll use the LLM to extract the key components
    analysis = anthropic_model.invoke(prompt)
    return parse_analysis(analysis.content)

def parse_analysis(analysis_text: str) -> Dict:
    """Parse the LLM's analysis into a structured format"""
    components = {
        "tables": [],
        "columns": [],
        "filters": [],
        "joins": [],
        "group_by": []
    }
    
    # Remove any "Explanation:" text
    analysis_text = analysis_text.split("Explanation:")[0].strip()
    
    # Parse each section
    current_section = None
    for line in analysis_text.split('\n'):
        line = line.strip()
        if not line:
            continue
            
        if line.startswith('Tables:'):
            current_section = 'tables'
            tables = line.split(':')[1].strip()
            components['tables'] = [t.strip() for t in tables.split(',') if t.strip()]
        elif line.startswith('Columns:'):
            current_section = 'columns'
            cols = line.split(':')[1].strip()
            for col_part in cols.split(','):
                col_part = col_part.strip()
                if col_part:
                    if '(' in col_part and ')' in col_part:
                        agg = col_part.split('(')[0].strip().upper()
                        col = col_part.split('(')[1].split(')')[0].strip()
                        components['columns'].append((agg, col))
                    else:
                        components['columns'].append((None, col_part))
        elif line.startswith('Filters:'):
            current_section = 'filters'
            filters = line.split(':')[1].strip()
            components['filters'] = [f.strip() for f in filters.split(',') if f.strip()]
        elif line.startswith('Joins:'):
            current_section = 'joins'
            joins = line.split(':')[1].strip()
            components['joins'] = [j.strip() for j in joins.split(',') if j.strip()]
        elif line.startswith('GroupBy:'):
            current_section = 'group_by'
            group_bys = line.split(':')[1].strip()
            components['group_by'] = [g.strip() for g in group_bys.split(',') if g.strip()]
        elif current_section:
            # Handle multi-line sections
            if current_section == 'tables':
                components['tables'].extend([t.strip() for t in line.split(',') if t.strip()])
            elif current_section == 'columns':
                for col_part in line.split(','):
                    col_part = col_part.strip()
                    if col_part:
                        if '(' in col_part and ')' in col_part:
                            agg = col_part.split('(')[0].strip().upper()
                            col = col_part.split('(')[1].split(')')[0].strip()
                            components['columns'].append((agg, col))
                        else:
                            components['columns'].append((None, col_part))
            elif current_section == 'filters':
                components['filters'].extend([f.strip() for f in line.split(',') if f.strip()])
            elif current_section == 'joins':
                components['joins'].extend([j.strip() for j in line.split(',') if j.strip()])
            elif current_section == 'group_by':
                components['group_by'].extend([g.strip() for g in line.split(',') if g.strip()])
    
    # Ensure schema is included in table names
    components['tables'] = [f"SPURCHASE.{t.split('.')[-1]}" if '.' not in t else t for t in components['tables']]
    
    # Ensure grouping columns are included in SELECT - CORRECTED VERSION
    for group_col in components['group_by']:
        # Check if this exact (None, group_col) pair exists
        col_exists = any(col == (None, group_col) for col in components['columns'])
        # Check if group_col appears in any non-aggregated column reference
        col_part_of_ref = any(group_col in col[1] for col in components['columns'] if col[0] is None)
        
        if not col_exists and not col_part_of_ref:
            components['columns'].append((None, group_col))
    
    return components

def parse_sparql_results(xml_response: str) -> List[Dict]:
    """Parse SPARQL XML results into a list of dictionaries"""
    try:
        root = ET.fromstring(xml_response)
        results = []
        
        for result in root.findall('.//{http://www.w3.org/2005/sparql-results#}result'):
            row = {}
            for binding in result:
                var_name = binding.attrib['name']
                value = binding[0]  # uri or literal
                if value.tag.endswith('uri'):
                    row[var_name] = value.text
                elif value.tag.endswith('literal'):
                    row[var_name] = value.text
            results.append(row)
        return results
    except ET.ParseError as e:
        print(f"Error parsing XML: {e}")
        return []
    
def generate_sql(components: Dict) -> str:
    """Generate clean SQL query from the analyzed components"""
    # Validate components
    if not components["tables"]:
        raise ValueError("No tables identified for SQL generation")
    
    # Clean all components first
    def clean_component(component):
        return component.replace('[', '').replace(']', '').strip()
    
    # Build SELECT clause - ensure GROUP BY columns are included
    select_parts = []
    
    # First add all GROUP BY columns to SELECT if they're not already there
    # for group_col in components.get("group_by", []):
    #     group_col = clean_component(group_col)
    #     if not any(col[1] == group_col for col in components["columns"] if col[0] is None):
    #         select_parts.append(group_col)
    
    # Then add the requested columns
    for agg, col in components["columns"]:
        col = clean_component(col)
        if not col:
            continue
        if agg:
            select_parts.append(f"{agg}({col})")
        else:
            if col not in select_parts:  # Don't add duplicates
                select_parts.append(col)
    
    if not select_parts:  # Default to all columns if none specified
        select_parts.append("*")

    select_clause = ", ".join(select_parts[1:])
    print("SELECT BEFORE "+select_clause)
    print(select_parts)
    
    # Build FROM clause
    from_table = clean_component(components["tables"][0])
    from_clause = from_table
    
    # Add joins only if they exist and are not empty
    join_clauses = []
    for join in components.get("joins", []):
        clean_join = clean_component(join)
        if clean_join and clean_join != 'INNER JOIN':
            join_clauses.append(f"INNER JOIN SPURCHASE.LFA1 ON {clean_join}")
    print("INNER JOIN "+clean_join)
    
    # Build WHERE clause
    where_clauses = []
    for filter_cond in components.get("filters", []):
        clean_filter = clean_component(filter_cond)
        if clean_filter:
            where_clauses.append(clean_filter)
    
    where_clause = " AND ".join(where_clauses) if where_clauses else ""
    where_clause = where_clause.replace(",", " AND")
    print("WHERE CLAUSE "+where_clause)
    
    # Build GROUP BY clause
    group_by_columns = [clean_component(g) for g in components.get("group_by", []) if clean_component(g)]
    group_by_clause = ", ".join(group_by_columns) if group_by_columns else ""
    
    # Construct the SQL
    sql = f"SELECT {select_clause} FROM {from_clause}"
    
    if join_clauses:
        sql += " " + " ".join(join_clauses)
    
    if where_clause:
        sql += f" WHERE {where_clause}"
    
    if group_by_clause:
        sql += f" GROUP BY {group_by_clause}"
    
    # Final formatting
    sql = sql.strip()
    if not sql.endswith(';'):
        sql += ';'
    
    return sql

def execute_sql(sql_query: str, conn) -> pd.DataFrame:
    """Execute the generated SQL query and return results"""
    cursor = conn.cursor()
    try:
        cursor.execute(sql_query)
        columns = [desc[0] for desc in cursor.description]
        rows = cursor.fetchall()
        return pd.DataFrame(rows, columns=columns)
    except Exception as e:
        print(f"Error executing SQL query: {e}")
        return pd.DataFrame()
    finally:
        cursor.close()


def process_question(question: str, conn, anthropic) -> pd.DataFrame:
    """Main function to process a user question with better error handling"""
    try:
        # Step 1: Extract relevant metadata using SPARQL
        metadata = extract_metadata(conn)
        
        if not metadata:
            return "Could not retrieve database metadata."
        
        # Step 2: Analyze the metadata and question
        components = analyze_metadata(metadata, question, anthropic)
        
        # Step 3: Generate SQL query
        sql_query = generate_sql(components)
    
        # Step 4: Execute SQL
        sql_result = execute_sql(sql_query, conn)
        
        return sql_result
    except Exception as e:
        return f"Error processing question: {str(e)}"

rdf_context = process_question(question, conn, anthropic_model)
print("\nKnowledge Graph Context Retrieved:")
print(rdf_context)

### Step 7: Generate Final Answer
By leveraging both sources simultaneously:
- The vector engine provides contextual richness and flexibility, helping the model understand the user's intent in a nuanced way.
- The knowledge graph ensures precision, factual consistency, and traceability by grounding answers in authoritative structured data.

In [ ]:
hybrid_prompt_template = PromptTemplate(
    template="""
        Context: You are tasked with helping retrieve and summarize supplier information.

        Available information:
        - Unstructured document chunks ({vector_context}).
        - Structured knowledge graph results ({kg_context}).

        User Question:
        {question}

        Instructions:
        - Only suppliers that appear in both the unstructured document chunks Structured knowledge graph results may be included in the final answer.
        - Use structured knowledge graph results to filter the suppliers already found in the unstructured document chunks.
        - Do not infer or assume facts — all conclusions must be backed by knowledge graph validation.
        - Optionally, include supporting information from unstructured data *if* it aligns with the knowledge graph result.
        - Prefer clarity and structured presentation (use lists or bullets). Optionally, return structured JSON if many suppliers are involved.

        Return:
        - A clean, human-readable summary limited to validated suppliers only.
        - Ensure all risk scores are grounded in validated graph entries.

        """,
    input_variables=["vector_context", "kg_context", "question"]
)
# Generate final answer using LLM
hybrid_answer_llm_chain = hybrid_prompt_template | gpt_model
hybrid_answer = hybrid_answer_llm_chain.invoke({
    "vector_context": vector_context,
    "kg_context": rdf_context,
    "question": question
}).content.strip()

print("\n=== Hybrid RAG Answer ===\n")
print(hybrid_answer)